In [9]:
import sys,os,argparse,time
import numpy as np
import torch
import utils
from datetime import datetime
import pickle
import pandas as pd
tstart=time.time()

In [10]:
# Arguments
parser=argparse.ArgumentParser(description='xxx')
parser.add_argument('--seed',               default=0,              type=int,     help='(default=%(default)d)')
parser.add_argument('--device',             default='cpu',          type=str,     help='gpu id')
parser.add_argument('--experiment',         default='16_task_groups',       type =str,    help='Mnist or dissertation')
parser.add_argument('--approach',           default='PUGCL',          type =str,    help='Method, always Lifelong Uncertainty-aware learning')
parser.add_argument('--data_path',          default='data/data_test.csv',     type=str,     help='gpu id')

# Training parameters
parser.add_argument('--output',             default='',             type=str,     help='')
parser.add_argument('--checkpoint_dir',     default='../checkpoints_16_tasks',    type=str,   help='')
parser.add_argument('--n_epochs',           default=100,              type=int,     help='')
parser.add_argument('--batch_size',         default=64,             type=int,     help='')
parser.add_argument('--lr',                 default=0.001,           type=float,   help='')
parser.add_argument('--hidden_size',        default=800,           type=int,     help='')
parser.add_argument('--parameter',          default='',             type=str,     help='')

# UCB HYPER-PARAMETERS
parser.add_argument('--MC_samples',         default='10',           type=int,     help='Number of Monte Carlo samples')
parser.add_argument('--rho',                default='-3',           type=float,   help='Initial rho')
parser.add_argument('--sigma1',             default='0.0',          type=float,   help='STD foor the 1st prior pdf in scaled mixture Gaussian')
parser.add_argument('--sigma2',             default='6.0',          type=float,   help='STD foor the 2nd prior pdf in scaled mixture Gaussian')
parser.add_argument('--pi',                 default='0.25',         type=float,   help='weighting factor for prior')

parser.add_argument('--resume',             default='no',           type=str,     help='resume?')
parser.add_argument('--sti',                default=1,              type=int,     help='starting task?')

parser.add_argument("-f", "--fff", help="a dummy argument to fool ipython", default="1")

args=parser.parse_args()
utils.print_arguments(args)

Arguments =
	seed: 0
	device: cpu
	experiment: 16_task_groups
	approach: PUGCL
	data_path: data/data_test.csv
	output: 
	checkpoint_dir: ../checkpoints_16_tasks
	n_epochs: 100
	batch_size: 64
	lr: 0.001
	hidden_size: 800
	parameter: 
	MC_samples: 10
	rho: -3.0
	sigma1: 0.0
	sigma2: 6.0
	pi: 0.25
	resume: no
	sti: 1
	fff: /Users/jonastjomsland/Library/Jupyter/runtime/kernel-f057205b-0a7b-4f32-bbd9-a98b1d5c68a1.json


In [11]:
# Set seed for stable results
np.random.seed(args.seed)
torch.manual_seed(args.seed)

# Check if Cuda is available
if torch.cuda.is_available():
    torch.cuda.manual_seed(args.seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

print("Using device:", args.device)

# Checkpoint
checkpoint = utils.make_directories(args)
args.checkpoint = checkpoint
print()

# PUGCL with two tasks:
from data import dataloader_16_tasks as dataloader

# Import Lifelong Uncertainty-aware Learning approach:
#from bayesian_model.lul import Lul
from training_method import PUGCL

# Import model used:
#from bayesian_model.bayesian_network import BayesianNetwork
from bayesian_model.bayesian_network import BayesianNetwork

Using device: cpu
16_task_groups_PUGCL
Results will be saved in  ../checkpoints_16_tasks/16_task_groups_PUGCL



In [12]:
# Load data:
print("Loading data...")
data, task_outputs, input_size = dataloader.get(data_path=args.data_path)
print("Input size =", input_size, "\nTask info =", task_outputs)
print("Number of data samples: ", len(data[0]['train']['x']))
args.num_tasks = len(task_outputs)
args.input_size = input_size
args.task_outputs = task_outputs

# Load features:
df_x1 = pd.read_csv('/Users/jonastjomsland/Dissertation/scripts_and_data/features4.csv')
df_x1 = df_x1.iloc[:, :-1]
df_x2 = pd.read_csv('/Users/jonastjomsland/Dissertation/data/features5.csv', sep=';')
df_x2.columns = df_x2.iloc[0]
df_x2 = df_x2.drop(df_x2.index[0])
df_x2 = df_x2.iloc[:, :-1]

features = pd.concat([df_x1, df_x2])
features = features.iloc[:-40,:-12]

Loading data...
Input size = [1, 29] 
Task info = [(0, 2), (1, 2), (2, 2), (3, 2), (4, 2), (5, 2), (6, 2), (7, 2), (8, 2), (9, 2), (10, 2), (11, 2), (12, 2), (13, 2), (14, 2), (15, 2)]
Number of data samples:  94


In [13]:
# Initialize Bayesian network
print("Initializing network...")
model = BayesianNetwork(args).to(args.device)

# Initialize Lul approach
print("Initialize Lifelong Uncertainty-aware Learning")
approach = Lul(model, args=args)
print("-"*100)

# Load stored model:
checkpoint = torch.load(os.path.join(args.checkpoint, 'model_{}.pth.tar'.format(args.sti)))
model.load_state_dict(checkpoint['model_state_dict'])
model = model.to(device=args.device)

Initializing network...
Initialize Lifelong Uncertainty-aware Learning


NameError: name 'Lul' is not defined

In [14]:
# Prediction and uncertainty:
# Get test data
# Circle or arrow:
arrow = 1 # If 0 then circle
xtest = data[arrow]['test']['x'].type(torch.float32).to(args.device)
ytest = data[arrow]['test']['y'].type(torch.float32).to(args.device)
# Get input and target
input_id = 2
input, target, = xtest[input_id:input_id+1,1:].to(args.device), ytest[0].to(args.device)
scene = int(xtest[input_id,0].detach().numpy())
if features.loc[features['Stamp'] == scene].empty:
    image_path = '/Users/jonastjomsland/Dissertation/' + features.loc[features['Stamp'] == str(scene)]['File Path'].values[0]
else:
    image_path = '/Users/jonastjomsland/Dissertation/' + features.loc[features['Stamp'] == scene]['File Path'].values[0]
    
questions = [['Vacuum cleaning', 'Mopping the floor', 'Carry warm food', 'Carry cold food', 'Carry drinks', 'Carry small objects (plates, toys)', 'Carry big objects (tables, chairs)', 'Cleaning (Picking up stuff)'], ['Vacuum cleaning', 'Mopping the floor', 'Carry warm food', 'Carry cold food', 'Carry drinks', 'Carry small objects (plates, toys)', 'Carry big objects (tables, chairs)', 'Starting conversation']]

# Do 100 MC samples:
mean = []
log_var = []

for sample in range(100):
    mean.append(model(input, sample = True)[arrow].detach().numpy()[0][0:8])
    log_var.append(model(input, sample = True)[arrow].detach().numpy()[0][8:])

Prediction = np.mean(mean, axis = 0)
Epistemic = np.mean(np.square(mean), axis = 0) - np.square(np.mean(mean, axis = 0))
Aleatoric = np.mean(np.exp(log_var), axis = 0)

IndexError: too many indices for tensor of dimension 1

In [15]:
i = 0
table = []
for question in questions[arrow]:
    table.append([question, Prediction[i], Epistemic[i], Aleatoric[i]])
    i += 1

NameError: name 'questions' is not defined

In [16]:
print("Scene number: ", scene)
img = mpimg.imread(image_path)
plt.figure(figsize = (10,12))
plt.rcParams["axes.grid"] = False
imgplot = plt.imshow(img)
plt.show()
print(tabulate(table, headers= ['Action', 'Prediction', 'Epistemic uncertainty', 'Aleatoric uncertainty'], tablefmt='orgtbl'))

NameError: name 'scene' is not defined

In [ ]:
# Prediction and uncertainty:
Prediction_arrow = []
Epistemic_arrow = []
Aleatoric_arrow = []
Prediction_circle = []
Epistemic_circle = []
Aleatoric_circle = []

# Circle or arrow:
arrow = 1 # If 0 then circle
for x_row in data[arrow]['test']['x'][0:200,:].type(torch.float32).to(args.device):
    # Get input and target
    x = x_row[1:].to(args.device).view(-1, 29)
    
    # Do 100 MC samples:
    mean = []
    log_var = []

    for sample in range(100):
        mean.append(model(x, sample = True)[arrow].detach().numpy()[0][0:8])
        log_var.append(model(x, sample = True)[arrow].detach().numpy()[0][8:])

    Prediction_arrow.append(np.mean(mean, axis = 0))
    Epistemic_arrow.append(np.mean(np.square(mean), axis = 0) - np.square(np.mean(mean, axis = 0)))
    Aleatoric_arrow.append(np.mean(np.exp(log_var), axis = 0))

# Circle or arrow:
arrow = 0 # If 0 then circle
for x_row in data[arrow]['test']['x'][0:200,:].type(torch.float32).to(args.device):
    # Get input and target
    x = x_row[1:].to(args.device).view(-1, 29)
    
    # Do 100 MC samples:
    mean = []
    log_var = []

    for sample in range(100):
        mean.append(model(x, sample = True)[arrow].detach().numpy()[0][0:8])
        log_var.append(model(x, sample = True)[arrow].detach().numpy()[0][8:])

    Prediction_circle.append(np.mean(mean, axis = 0))
    Epistemic_circle.append(np.mean(np.square(mean), axis = 0) - np.square(np.mean(mean, axis = 0)))
    Aleatoric_circle.append(np.mean(np.exp(log_var), axis = 0))

In [17]:
i = 0
table = []
for question in questions[0]:
    table.append([question, np.mean(np.array(Prediction_arrow), axis = 0)[i], np.mean(np.array(Epistemic_arrow), axis = 0)[i], np.mean(np.array(Aleatoric_arrow), axis = 0)[i]])
    i += 1
print("Results for Circle (50 sample average): ")
print(tabulate(table, headers= ['Action', 'Prediction', 'Epistemic uncertainty', 'Aleatoric uncertainty'], tablefmt='orgtbl'))
i = 0
table = []
for question in questions[1]:
    table.append([question, np.mean(np.array(Prediction_circle), axis = 0)[i], np.mean(np.array(Epistemic_circle), axis = 0)[i], np.mean(np.array(Aleatoric_circle), axis = 0)[i]])
    i += 1
print("Results for Arrow (50 sample average): ")
print(tabulate(table, headers= ['Action', 'Prediction', 'Epistemic uncertainty', 'Aleatoric uncertainty'], tablefmt='orgtbl'))


NameError: name 'questions' is not defined

In [18]:
# image printer:
scene = 667
if features.loc[features['Stamp'] == scene].empty:
    image_path = '/Users/jonastjomsland/Dissertation/' + features.loc[features['Stamp'] == str(scene)]['File Path'].values[0]
else:
    image_path = '/Users/jonastjomsland/Dissertation/' + features.loc[features['Stamp'] == scene]['File Path'].values[0]

print("Scene number: ", scene)
img = mpimg.imread(image_path)
plt.figure(figsize = (15,20))
plt.rcParams["axes.grid"] = False
imgplot = plt.imshow(img)
plt.show()

Scene number:  667


NameError: name 'mpimg' is not defined

In [16]:
print('/Users/jonastjomsland/Dissertation/' + features.loc[features['Stamp'] == stamp]['File Path'])

Series([], Name: File Path, dtype: object)
